![alt text](./img/header.png)

# Saltwater Intrusion Exercise

A growing coastal city is interested in developing a wellfield in order to provide potable drinking water to its residents.  You have been contracted by the city to perform a numerical analysis to determine the effects of groundwater withdrawals on the position of the saltwater interface.  The city is particularly interested in knowing if the planned withdrawal rate will result in pumped water that exceeds the drinking water standard for chloride, which is 250 mg/L.  In addition, you have been asked by the city to delineate a wellfield protection zone by determining the approximate recharge area for the wellfield.  The city is also interested in knowing what would happen if a spill was to occur at the airport.  At what concentrations would a conservative contaminant plume reach the proposed wellfield?

![alt text](./img/exB_fig1.png)

The study area consists of a 10 by 10 km area of a coastal plain aquifer system centered on the proposed wellfield (Figure 1).  The hydrogeology of the area consists of a shallow and deep aquifer, and both aquifers intersect the sea floor (Figure 2).  The two aquifers are separated by a discontinuous confining layer that thickens to the east.  The confining layer is present in the eastern part of the study area, but is absent in the western part.  A low permeability unit, which slopes downward to the east, underlies the deep aquifer and can be considered a no flow boundary.  Extensive field investigations have been performed, and the aquifer properties for the different units are summarized in the table below.  Annual average recharge was determined to be 25.4 cm/yr (10 in/yr).

![alt text](./img/exB_fig2.png)

Unfortunately, due to limitations in available property, the city has only one option for wellfield location, which is shown on the map in Figure 1.  The city is hoping to capture at least 25% of the recharge for the area, but there is an obvious concern that excessive pumping from the deep aquifer could cause the saltwater interface to move inland and contaminate the wellfield.

  Unit            | $K_h$ |  $K_v$      |  $S$        | $S_y$ | $n$   | $\alpha_l$ | $\alpha_v$ 
  --------------- | ----- | ----------- | ----------- | ----- | ----- | ---------- | ---------- 
  Shallow aquifer | 100   | 1           | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Confining unit  | 0.001 | 0.001       | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          
  Deep aquifer    | 2000  | 200         | $1x10^{-5}$ | 0.2   | 0.2   | 10         | 1          



In [ ]:
# Setup the python environment
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

## Exercise C -- Design, run, and calibrate a 2D cross-section model to obtain the steady-state pre-withdrawal distribution of head and salinity

Based on the geometry of the system, the pre-withdrawal conditions can be obtained by running a 2D cross-section model.  The resulting heads and salinities can then be used as initial conditions for the 3D model to evaluate interface movement in response to pumping.  Because 3D saltwater intrusion models can take a long time, across-sectional model is developed first using one row, 100 columns, and 25 model layers. The model has already been constructed for you.

This exercise was designed such that a 2D model could be used to represent pre-withdrawal conditions.  If the hydrogeology were to vary in the north-south direction, or if an irregular boundary were to exist, the problem of obtaining equilibrium conditions would be more difficult.

### Part C1 -- Determine length of simulation period required to reach equilibrium

In [ ]:
# Data path
datapth = os.path.join('data', 'exSEAWAT_B')

# Grid information
nlay = 23
nrow = 1
ncol = 100
delr = 100.
delc = 100.
top = 2.5
botm = np.linspace(-2.5, -112.5, nlay)

# Temporal discretization
nper = 1
nstp = 10000
perlen = 100000.

# Ibound
fname = os.path.join(datapth, 'ibound.txt')
ibound = np.loadtxt(fname).reshape((nlay, nrow, ncol))

# Hydraulic properties
fname = os.path.join(datapth, 'hk.txt')
hk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
fname = os.path.join(datapth, 'vk.txt')
vk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
ss = 1.e-5
sy = 0.2

In [ ]:
ksel = [100., 0.001, 2000.]
kvsel = [1., 0.001, 200.]
hkc = hk.copy()
for jdx, kk in enumerate(ksel):
    idx = np.abs(hkc - kk) < 1e-6
    hkc[idx] = jdx + 1
for k in range(nlay - 1):
    for i in range(ncol):
        if hkc[k, 0, i] == 1 and hkc[k+1, 0, i] == 3:
            hkc[k+1, 0, i] = 4
idx = (ibound[:, 0, :80] == 0) & (hkc[:, 0, :80] == 3)
hkc[:, 0, :80][idx] = 0
botms = np.zeros((3, 1, hkc.shape[2]))

# k and kv for swi model
kswi = np.zeros((3, 1, hkc.shape[2]))
kvswi = np.zeros((3, 1, hkc.shape[2]))

kswi[0, 0, :] = ksel[0]
kswi[2, 0, :] = ksel[2]
kvswi[0, 0, :] = kvsel[0]
kvswi[2, 0, :] = kvsel[2]
for i in range(ncol):
    for k in range(nlay - 1):
        if hkc[k, 0, i] == 1 and (hkc[k+1, 0, i] == 4 or hkc[k+1, 0, i] == 2):
            botms[0, 0, i] = botm[k]
        if (hkc[k, 0, i] == 4 or hkc[k, 0, i] == 2) and hkc[k+1, 0, i] == 3:
            botms[1, 0, i] = botm[k]
            if hkc[k, 0, i] == 4:
                kswi[1, 0, i] = ksel[2]
                kvswi[1, 0, i] = kvsel[2]
            else:
                kswi[1, 0, i] = ksel[1]
                kvswi[1, 0, i] = kvsel[1]
        if hkc[k, 0, i] == 3 and hkc[k+1, 0, i] == 0:
            botms[2, 0, i] = botm[k]
        if hkc[k, 0, i] == 3 and hkc[k+1, 0, i] == 3:
            botms[2, 0, i] = botm[k+1]
#print(botms[0, 0, :])
#print(botms[1, 0, :])
#print(botms[2, 0, :])
#print(kswi[:, 0, :])
t = np.ones(ncol) * delr
xc = np.cumsum(t) - delr / 2.
#print(xc)

In [ ]:
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
q = ax.imshow(hkc[:, 0, :], interpolation='none', extent=[0, 10000, botm[-1], 2.5], aspect=20)
plt.colorbar(q, shrink=0.5)
plt.plot(xc, botms[0, 0, :], color='cyan', drawstyle='steps-mid', lw=2)
plt.plot(xc, botms[1, 0, :], color='cyan', drawstyle='steps-mid', lw=2)
plt.plot(xc, botms[2, 0, :], color='cyan', drawstyle='steps-mid', lw=2);

In [ ]:
topswi = np.ones(ncol) * 2.5
ibswi = np.ones((3, 1, hkc.shape[2]), dtype=np.int)
for i in range(ncol):
    for k in range(nlay - 1):
        if ibound[k, 0, i] == 0 and ibound[k+1, 0, i] < 0:
            topswi[i] = botm[k]
zmin = 0.1
for i in range(ncol):
    if botms[0, 0, i] >= topswi[i]:
        botms[0, 0, i] = topswi[i] - zmin
for i in range(ncol):
    if botms[1, 0, i] >= botms[0, 0, i]:
        botms[1, 0, i] = botms[0, 0, i] - zmin 

dz = np.ones((3, 1, hkc.shape[2]))
for k in range(3):
    for i in range(ncol):
        if k == 0:
            z1 = topswi[i]
        else:
            z1 = botms[k-1, 0, i]
        dz[k, 0, i] = z1 - botms[k, 0, i]
        if abs(dz[k, 0, i]- zmin) < 1.0e-6:
            ibswi[k, 0, i] = 0
#print(dz[:, 0, :])

for i in range(ncol):
    if topswi[i] < 2.5 and ibswi[0, 0, i] > 0:
        ibswi[0, 0, i] = 2
for i in range(ncol):
    if botms[0, 0, i] < -47.5 and ibswi[1, 0, i] > 0:
        ibswi[1, 0, i] = 2
for i in range(ncol):
    if botms[1, 0, i] < -72.5 and ibswi[2, 0, i] > 0:
        ibswi[2, 0, i] = 2
        
#print(ibswi[:, 0, :])

In [ ]:
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
ax.fill_between(xc, y1=topswi, y2=botms[0, 0, :], color='cyan', step='mid')
ax.fill_between(xc, y1=botms[0, 0, :], y2=botms[1, 0, :], color='brown', step='mid')
ax.fill_between(xc, y1=botms[1, 0, :], y2=botms[2, 0, :], color='blue', step='mid');

In [ ]:
# save data
datapth = os.path.join('data', 'exSEAWAT_B-SWI')
# top
fn = os.path.join(datapth, 'top.txt')
np.savetxt(fn, topswi.reshape(1, ncol))
# botms
fn = os.path.join(datapth, 'botm.txt')
np.savetxt(fn, botms[:, 0, :])
# ibswi
fn = os.path.join(datapth, 'ibound.txt')
np.savetxt(fn, ibswi[:, 0, :])
# hk
fn = os.path.join(datapth, 'hk.txt')
np.savetxt(fn, kswi[:, 0, :])
# vk
fn = os.path.join(datapth, 'vk.txt')
np.savetxt(fn, kvswi[:, 0, :])


In [ ]:
# Recharge
rech = 25.4 / 100. / 365.  # convert cm/yr to m/day

# Build ghbs
nghb = 0
for k in range(3):
    for j in range(ncol):
        if ibswi[k, 0, j] == 2:
            nghb += 1
ghb_data = flopy.modflow.ModflowGhb.get_empty(ncells=nghb)
idx = 0
for k in range(3):
    for j in range(ncol):
        if ibswi[k, 0, j] == 2:
            if k == 0:
                z1 = min(0., topswi[j])
            else:
                z1 = botms[k-1, 0, j]
            hfw = 0.025 * (-z1)
            d = (z1 - botms[k, 0, j])
            cond = kswi[k, 0, j] * d * delr / (0.5 * delc)
            ghb_data['k'][idx] = k
            ghb_data['i'][idx] = 0
            ghb_data['j'][idx] = j
            ghb_data['bhead'][idx] = hfw
            ghb_data['cond'][idx] = cond
            idx += 1
#print(ghb_data.dtype)
print(ghb_data)

In [ ]:
# Swi
isource = np.zeros((3, 1, hkc.shape[2]), dtype=np.int)
for k in range(3):
    for j in range(ncol):
        if ibswi[k, 0, j] == 2:
            isource[k, 0, j] = -2

# assume freshwater to coastline and saltwater beyond
ipos = 0
for j in range(ncol):
    if topswi[j] < 2.5:
        ipos = j
        break
zeta = np.ones((3, 1, ncol)) * -250.
zeta[:, :, ipos:] = 2.5

In [ ]:
# Build the flopy SEAWAT model
model_ws = os.path.join('work', 'exSEAWAT_B-SWI')
if not os.path.isdir(model_ws):
    os.mkdir(model_ws)
    
modelname = 'b1-swi'
m = flopy.modflow.Modflow(modelname, model_ws=model_ws, exe_name=config.mfexe)
dis = flopy.modflow.ModflowDis(m, nlay=3, nrow=nrow, ncol=ncol, delr=delr, delc=delc, 
                               top=topswi.reshape(1, ncol), botm=botms, perlen=perlen, nstp=nstp)
bas = flopy.modflow.ModflowBas(m, ibswi, strt=0.)
lpf = flopy.modflow.ModflowLpf(m, laytyp=1, hk=kswi, vka=kvswi, ss=ss, sy=sy)
rch = flopy.modflow.ModflowRch(m, rech=rech)
ghb = flopy.modflow.ModflowGhb(m, stress_period_data=ghb_data)
swi = flopy.modflow.ModflowSwi2(m, ssz=0.2, zeta=[zeta], isource=isource, nsrf=1)
oc = flopy.modflow.ModflowOc(m, stress_period_data={(0, nstp-1):['save head']})
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-3, rclose=1e4)

In [ ]:
# Make plot of the grid
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()

In [ ]:
# Make color flood plot of hydraulic conductivity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
hkpatchcollection = mm.plot_array(np.log(m.lpf.hk.array), cmap='viridis')
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()
cb = plt.colorbar(hkpatchcollection)

In [ ]:
# write the input files
m.write_input()

In [ ]:
# run the model
success, buff = m.run_model(silent=True)
print(success)

In [ ]:
# Extract salinity
fname = os.path.join(model_ws, 'b1-swi.zta')
zobj = flopy.utils.binaryfile.CellBudgetFile(fname)
print(zobj.list_records())
kk = zobj.get_kstpkper()
zeta = zobj.get_data(text='ZETASRF  1', kstpkper=kk[-1])[0]
#print(zeta)

In [ ]:
# Make plot of the simulated salinity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
mm.plot_fill_between(zeta);
#cpatchcollection = mm.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
linecollection = mm.plot_grid()
#patchcollection = mm.plot_ibound()
#cb = plt.colorbar(cpatchcollection)